In [83]:
import os, json, re    

In [84]:
def importJSONFiles(files: list, path: str) -> list[dict[str, any]]:

    hts_data = []

    for file in files:

        with open(f'{path}/{file}') as f:
            header = re.sub(r'\.json', '', file)
            hts_data.append(
                {
                    'header': header,
                    'data': json.loads(f.read())
                })

    return hts_data

def getHTSRecord(hts_data: list[dict[str, any]], queryStr: str):

    for record in hts_data:
        
        if record['header'] == queryStr:
            return record['data']
    
    return 'No record found'

In [85]:
def checkResultQuery(result_query: list[dict[str, any]]):

    index = 0
    indent_list = [record['indent'] for record in result_query if 'indent' in record]
    
    for i in range(0, result_query[-1]['indent']):
        
        if i not in indent_list:
    
            if 'missing' in result_query[index-1]:

                result_query[index-1]['missing'].append(i)
            else:
                result_query[index-1]['missing'] = [i]

        else:
            index += 1

def searchEHIndents(result_query: list[dict[str, any]], hts_record: list[dict[str, any]]):

    result_final = []

    for key, result in enumerate(result_query):

        if 'missing' not in result: continue

        next_hts = result_query[key+1]['htsno']
        indents = result_query[key]['missing']

        for i in range(result['indexHTSRec'], result_query[key+1]['indexHTSRec']):

            for indent in indents:

                if 'htsno' not in hts_record[i] and indent == hts_record[i]['indent']:
                    print(hts_record[i])
                    print(i)
                    result_final.append(hts_record[i])

    def iterateHTS(hts_record: list[dict[str, any]], next_hts: str, start: int, end: int):

        result = []
        subtractor = 0

        for i in range(start, end):

            subtractor += 1

            if 'htsno' in hts_record[i] and hts_record[i] == next_hts:
                return i - subtractor

    return result_final



In [86]:
def grabQueryRecords(hts_record: list[dict[str, any]], query: list[str]) -> list[dict[str, any]]:

    result = []
    index_query = 0

    while index_query < len(query):

        for key, record in enumerate(hts_record):

            if 'htsno' in record and re.match(rf'{query[index_query]}$', record['htsno']):

                result.append({
                    'htsno': record['htsno'],
                    'indent': record['indent'],
                    'description': record['description'],
                    'indexHTSRec': key
                })

        index_query += 1
    
    checkResultQuery(result)
    
    return result

In [87]:
path = '../../db_hts/temp/NEW_final_json_files'
query = ["0201", "0201.20", "0201.20.06", "0201.20.06.00"]
files = os.listdir(path)
hts_data = importJSONFiles(files, path)

In [88]:
hts_record = getHTSRecord(hts_data, query[0])

In [89]:
result = grabQueryRecords(hts_record, query)
result

[{'htsno': '0201',
  'indent': 0,
  'description': 'Meat of bovine animals, fresh or chilled:',
  'indexHTSRec': 0},
 {'htsno': '0201.20',
  'indent': 1,
  'description': 'Other cuts with bone in:',
  'indexHTSRec': 11,
  'missing': [2]},
 {'htsno': '0201.20.06.00',
  'indent': 3,
  'description': 'Other',
  'indexHTSRec': 16}]

In [90]:
EHrecords = searchEHIndents(result, hts_record)
EHrecords

{'indent': 2, 'description': 'Described in general note 15 of the tariff schedule and entered pursuant to its provisions:', 'superior': 'true'}
12


[{'indent': 2,
  'description': 'Described in general note 15 of the tariff schedule and entered pursuant to its provisions:',
  'superior': 'true'}]